In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

import season1.statistician as stat
import season1.utility as util
from season1.validater import get_label, _prepare_level, _is_playable

def transform_data_block_to_compared(data_block):
    return {
        "map": data_block["map"],
        "example_maps": data_block["examples"],
        "before_params": data_block["target_params"],
        "after_params": get_label(data_block["map"])
    }

def generate_filename_from_used_paramname(used_paramname, recursion_count):
    filename = ""
    for param in used_paramname:
        filename += param
        filename += "_"
    filename += str(recursion_count)
    return filename

def calc_controllability(compared_list, used_paramname):
    controllability = {}
    
    for paramname in used_paramname:
        values = []
        
        for compared in compared_list:
            before = compared["before_params"][paramname]
            after = compared["after_params"][paramname]
            
            try:
                values.append(1 - (abs(before - after) / max(before, after)))
            except:
                pass      
        controllability[paramname] = np.average(values)
    
    return controllability

def is_playable(level: str) -> bool:
    list_level = _prepare_level(level)
    return _is_playable(level)

In [ ]:
result_dir = os.path.join("..", "data", "result")
analysis_dir = os.path.join("..", "data", "analysis")
paramname_list = ["map_size", "room_count", "enemy_count", "treasure_count", "exploration", "winding_path"]
recursion_count = 5

In [ ]:
controllability_data = {} # use like controllability_data["map_size_room_count"]["map_size"][5]

for paramname in paramname_list:
    used_paramname = [paramname]
    base_filename = "_".join(used_paramname)
    controllability_data[base_filename] = {}
    for i in range(recursion_count + 1):
        filename = "_".join([base_filename, str(i)])
        try:
            data_blocks = util.read_json_file(os.path.join(result_dir, filename))
        except:
            print(f"{filename} was skipped.")
            continue
        else:
            print(f"{filename} OK")

        compared_list = list(map(transform_data_block_to_compared, data_blocks))
        controllability = calc_controllability(compared_list, used_paramname)
        
        for paramname, value in controllability.items():
            if paramname not in controllability_data[base_filename]:
                controllability_data[base_filename][paramname] = []
                
            controllability_data[base_filename][paramname].append(value)

In [ ]:
plt.figure(figsize=(10, 6))

for paramname in paramname_list:
    x = ["base"] + list(range(1, recursion_count + 1))
    values = controllability_data[paramname][paramname]

    plt.plot(x, values, marker='o', label=paramname)

plt.title('Controllability When One Parameter Is Used')
plt.xlabel('Recursion Step')
plt.ylabel('Controllability Score')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd

data = {
    'Recursion Step': ["base"] + list(range(1, recursion_count + 1))
}

for paramname in paramname_list:
    values = controllability_data[paramname][paramname]
    data[paramname] = values

df = pd.DataFrame(data)

util.create_directory(analysis_dir)
df.to_csv(os.path.join(analysis_dir, "one_parameter_used.csv"), index=False)

In [ ]:
used_paramname_set = set()

for i in range(len(paramname_list)):
    for j in range(i + 1, len(paramname_list)):
        used_paramname_set.add((paramname_list[i], paramname_list[j]))

for used_paramname in used_paramname_set:
    base_filename = "_".join(used_paramname)
    controllability_data[base_filename] = {}
    
    for i in range(recursion_count + 1):
        filename = "_".join([base_filename, str(i)])
        try:
            data_blocks = util.read_json_file(os.path.join(result_dir, filename))
        except:
            print(f"{filename} was skipped.")
            continue
        else:
            print(f"{filename} OK")

        compared_list = list(map(transform_data_block_to_compared, data_blocks))
        controllability = calc_controllability(compared_list, used_paramname)
        
        for paramname, value in controllability.items():
            if paramname not in controllability_data[base_filename]:
                controllability_data[base_filename][paramname] = []
                
            controllability_data[base_filename][paramname].append(value)

In [ ]:
plt.figure(figsize=(10, 6))

data = {
    'Recursion Step': ["base"] + list(range(1, recursion_count + 1))
}

for paramname in paramname_list:
    x = ["base"] + list(range(1, recursion_count + 1))
    
    values = []
    for used_paramname in used_paramname_set:
        if paramname in used_paramname:
            values.append(controllability_data["_".join(used_paramname)][paramname])

    values = np.mean(values, axis=0)
    
    data[paramname] = values

    plt.plot(x, values, marker='o', label=paramname)

plt.title('Controllability When Two Parameter Are Used')
plt.xlabel('Recursion Step')
plt.ylabel('Controllability Score')
plt.legend()
plt.grid(True)
plt.show()

df = pd.DataFrame(data)

util.create_directory(analysis_dir)
df.to_csv(os.path.join(analysis_dir, "two_parameter_used.csv"), index=False)

In [ ]:
base_filename = "_".join(paramname_list)
controllability_data[base_filename] = {}

for i in range(recursion_count + 1):
    filename = "_".join([base_filename, str(i)])
    try:
        data_blocks = util.read_json_file(os.path.join(result_dir, filename))
    except:
        print(f"{filename} was skipped.")
        continue
    else:
        print(f"{filename} OK")

    compared_list = list(map(transform_data_block_to_compared, data_blocks))
    controllability = calc_controllability(compared_list, paramname_list)
    
    for paramname, value in controllability.items():
        if paramname not in controllability_data[base_filename]:
            controllability_data[base_filename][paramname] = []
            
        controllability_data[base_filename][paramname].append(value)

In [ ]:
plt.figure(figsize=(10, 6))

data = {
    'Recursion Step': ["base"] + list(range(1, recursion_count + 1))
}

base_filename = "_".join(paramname_list)
for paramname in paramname_list:
    x = ["base"] + list(range(1, recursion_count + 1))
    
    values = controllability_data[base_filename][paramname]
    data[paramname] = values
    
    plt.plot(x, values, marker='o', label=paramname)

plt.title('Controllability When Six Parameter Are Used')
plt.xlabel('Recursion Step')
plt.ylabel('Controllability Score')
plt.legend()
plt.grid(True)
plt.show()

df = pd.DataFrame(data)

util.create_directory(analysis_dir)
df.to_csv(os.path.join(analysis_dir, "six_parameter_used.csv"), index=False)

In [ ]:
playability_data = {} # use like playability_data["map_size"][5]

for paramname in paramname_list:
    used_paramname = [paramname]
    base_filename = "_".join(used_paramname)
    playability_data[base_filename] = []
    for i in range(recursion_count + 1):
        filename = "_".join([base_filename, str(i)])
        try:
            data_blocks = util.read_json_file(os.path.join(result_dir, filename))
        except:
            print(f"{filename} was skipped.")
            continue
        else:
            print(f"{filename} OK")

        playability_list = [is_playable(data_block["map"]) for data_block in data_blocks]
        
        playability_data[base_filename].append(np.mean(playability_list))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

plt.figure(figsize=(10, 6))

csv_data = {
    'Recursion Step': ["base"] + list(range(1, recursion_count + 1))
}

for paramname, scores in playability_data.items():
    plt.plot(csv_data['Recursion Step'], scores, marker='o', label=paramname)
    csv_data[paramname] = scores

plt.title('Playability Analysis Across Parameters')
plt.xlabel('Recursion Step')
plt.ylabel('Playability Score')
plt.legend()
plt.grid(True)
plt.show()

df = pd.DataFrame(csv_data)
df.to_csv(os.path.join(analysis_dir, "playability_analysis.csv"), index=False)